In [83]:
import pandas as pd
import plotly.graph_objs as go 
import matplotlib.pyplot as plt

In [84]:
apple = pd.read_csv("../data/AAPL.csv")
amazon = pd.read_csv("../data/AMZN.csv")
berkshire = pd.read_csv("../data/BRK-B.csv")
google = pd.read_csv("../data/GOOG.csv")
meta = pd.read_csv("../data/META.csv")
microsoft = pd.read_csv("../data/MSFT.csv")
nvidia = pd.read_csv("../data/NVDA.csv")
tesla = pd.read_csv("../data/TSLA.csv")
unitedhealth = pd.read_csv("../data/UNH.csv")
exxon = pd.read_csv("../data/XOM.csv")

In [86]:
df_names = ['apple', 'amazon', 'berkshire', 'google', 'meta', 'microsoft', 'nvidia', 'tesla', 'unitedhealth', 'exxon']
merged_df = globals()[df_names[0]][['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

for df_name in df_names[1:]:
    merged_df = pd.merge(merged_df, globals()[df_name], on='Date', how='outer', suffixes=('', f'_{df_name}'))


cours_boursiers = merged_df.rename(columns={"Open": "Open_apple",
                                      "High": "High_apple",
                                      "Low": "Low_apple",
                                      "Close": "Close_apple",
                                      "Adj Close": "Adj Close_apple",
                                      "Volume":"Volume_apple"})


cours_boursiers.columns = cours_boursiers.columns.str.replace(" ","_")
cours_boursiers.head()

,Date,Open_apple,High_apple,Low_apple,Close_apple,Adj_Close_apple,Volume_apple,Open_amazon,High_amazon,Low_amazon,...,Low_unitedhealth,Close_unitedhealth,Adj_Close_unitedhealth,Volume_unitedhealth,Open_exxon,High_exxon,Low_exxon,Close_exxon,Adj_Close_exxon,Volume_exxon
0,2013-01-02,19.779285,19.821428,19.343929,19.608213,16.769093,560518000,12.8040,12.9050,12.6630,...,54.009998,54.540001,46.073730,5358100,87.790001,88.720001,87.239998,88.709999,56.165756,16143700
1,2013-01-03,19.567142,19.631071,19.321428,19.360714,16.557426,352965200,12.8635,13.0440,12.8185,...,51.849998,51.990002,43.919567,14413200,88.459999,88.889999,88.160004,88.550003,56.064438,13268200
2,2013-01-04,19.177500,19.236786,18.779642,18.821428,16.096228,594333600,12.8790,12.9900,12.8325,...,52.060001,52.090000,44.004032,10458000,88.500000,89.089996,88.370003,88.959999,56.324017,11427900
3,2013-01-07,18.642857,18.903570,18.400000,18.710714,16.001543,484156400,13.1485,13.4865,13.1335,...,51.549999,52.090000,44.004032,7804500,88.309998,88.489998,87.570000,87.930000,55.671913,11799800
4,2013-01-08,18.900356,18.996071,18.616072,18.761070,16.044611,458707200,13.3535,13.4490,13.1785,...,51.360001,51.400002,43.421143,8196200,87.820000,88.480003,87.290001,88.480003,56.020119,14226400


In [87]:
cours_boursiers.columns

Index(['Date', 'Open_apple', 'High_apple', 'Low_apple', 'Close_apple',
       'Adj_Close_apple', 'Volume_apple', 'Open_amazon', 'High_amazon',
       'Low_amazon', 'Close_amazon', 'Adj_Close_amazon', 'Volume_amazon',
       'Open_berkshire', 'High_berkshire', 'Low_berkshire', 'Close_berkshire',
       'Adj_Close_berkshire', 'Volume_berkshire', 'Open_google', 'High_google',
       'Low_google', 'Close_google', 'Adj_Close_google', 'Volume_google',
       'Open_meta', 'High_meta', 'Low_meta', 'Close_meta', 'Adj_Close_meta',
       'Volume_meta', 'Open_microsoft', 'High_microsoft', 'Low_microsoft',
       'Close_microsoft', 'Adj_Close_microsoft', 'Volume_microsoft',
       'Open_nvidia', 'High_nvidia', 'Low_nvidia', 'Close_nvidia',
       'Adj_Close_nvidia', 'Volume_nvidia', 'Open_tesla', 'High_tesla',
       'Low_tesla', 'Close_tesla', 'Adj_Close_tesla', 'Volume_tesla',
       'Open_unitedhealth', 'High_unitedhealth', 'Low_unitedhealth',
       'Close_unitedhealth', 'Adj_Close_unitedhealt

## Feature engineering

In [90]:

cours_boursiers['Date'] = pd.to_datetime(cours_boursiers['Date'])

cours_boursiers = cours_boursiers.sort_values(by='Date')
stock_columns = ['apple', 'amazon', 'berkshire', 'google', 'meta', 'microsoft', 'nvidia', 'tesla', 'unitedhealth', 'exxon']

for stock in stock_columns:
    # Calcul de la volatilité High/Low sur 10 jours
    cours_boursiers[f'Volatility_HighLow_{stock}_10days'] = cours_boursiers[f'High_{stock}'].rolling(window=10).std() / cours_boursiers[f'Low_{stock}'].rolling(window=10).mean()

    # Calcul du delta High/Low sur 10 jours
    cours_boursiers[f'Delta_HighLow_{stock}_10days'] = cours_boursiers[f'High_{stock}'].diff(periods=10) / cours_boursiers[f'Low_{stock}'].diff(periods=10)

    # Calcul du delta Open/Close sur 10 jours
    cours_boursiers[f'Delta_OpenClose_{stock}_10days'] = cours_boursiers[f'Open_{stock}'].diff(periods=10) / cours_boursiers[f'Close_{stock}'].diff(periods=10)

    # Calcul du delta de volume sur 10 jours
    cours_boursiers[f'Delta_Volume_{stock}_10days'] = cours_boursiers[f'Volume_{stock}'].diff(periods=10)


In [93]:
cours_boursiers.shape

(2768, 101)

In [97]:
import pandas as pd

# Supposons que votre DataFrame soit déjà chargé avec les données
# cours_boursiers = ...

# Liste des colonnes représentant différentes actions
stock_columns = ['apple', 'amazon', 'berkshire', 'google', 'meta', 'microsoft', 'nvidia', 'tesla', 'unitedhealth', 'exxon']

# Prétraitement des séries temporelles pour chaque action
for stock in stock_columns:
    cours_boursiers[f"{stock}_return"] = cours_boursiers[f"Close_{stock}"].pct_change()

# Création d'un nouveau DataFrame pour stocker les rendements de chaque action et le rendement global du portefeuille
portfolio_returns = cours_boursiers[[f"{stock}_return" for stock in stock_columns]].copy()

# Calcul du rendement global du portefeuille
portfolio_returns['Portfolio'] = portfolio_returns.mean(axis=1)

# Initialisation du capital initial
initial_capital = 1000

# Calcul de la valeur du portefeuille
portfolio_returns['Portfolio Value'] = initial_capital * (1 + portfolio_returns['Portfolio']).cumprod()

# Suppression de la première ligne, car les rendements ne sont pas définis pour la première observation
portfolio_returns = portfolio_returns.iloc[1:]

# Ajout des colonnes de rendements au DataFrame d'origine
cours_boursiers = pd.concat([cours_boursiers, portfolio_returns], axis=1)

# Afficher les premières lignes du DataFrame résultant
cours_boursiers

,Date,Open_apple,High_apple,Low_apple,Close_apple,Adj_Close_apple,Volume_apple,Open_amazon,High_amazon,Low_amazon,...,nvidia_return,nvidia_return,tesla_return,tesla_return,unitedhealth_return,unitedhealth_return,exxon_return,exxon_return,Portfolio,Portfolio Value
0,2013-01-02,19.779285,19.821428,19.343929,19.608213,16.769093,560518000,12.804000,12.905000,12.663000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-03,19.567142,19.631071,19.321428,19.360714,16.557426,352965200,12.863500,13.044000,12.818500,...,0.000786,0.000786,-0.016685,-0.016685,-0.046755,-0.046755,-0.001804,-0.001804,-0.008906,991.094425
2,2013-01-04,19.177500,19.236786,18.779642,18.821428,16.096228,594333600,12.879000,12.990000,12.832500,...,0.032993,0.032993,-0.010642,-0.010642,0.001923,0.001923,0.004630,0.004630,0.004279,995.335697
3,2013-01-07,18.642857,18.903570,18.400000,18.710714,16.001543,484156400,13.148500,13.486500,13.133500,...,-0.028897,-0.028897,-0.001744,-0.001744,0.000000,0.000000,-0.011578,-0.011578,0.000028,995.363207
4,2013-01-08,18.900356,18.996071,18.616072,18.761070,16.044611,458707200,13.353500,13.449000,13.178500,...,-0.021926,-0.021926,-0.019220,-0.019220,-0.013246,-0.013246,0.006255,0.006255,-0.006880,988.515396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,2023-12-22,195.179993,195.410004,192.970001,193.600006,193.600006,37122800,153.770004,154.350006,152.710007,...,-0.003266,-0.003266,-0.007701,-0.007701,0.000827,0.000827,0.001769,0.001769,-0.000843,18336.176369
2764,2023-12-26,193.610001,193.889999,192.830002,193.050003,193.050003,28919300,153.559998,153.979996,153.029999,...,0.009195,0.009195,0.016116,0.016116,-0.000538,-0.000538,0.002257,0.002257,0.003012,18391.410561
2765,2023-12-27,192.490005,193.500000,191.089996,193.149994,193.149994,48087700,153.559998,154.779999,153.119995,...,0.002800,0.002800,0.018822,0.018822,0.005307,0.005307,-0.004699,-0.004699,0.001985,18427.910384
2766,2023-12-28,194.139999,194.660004,193.169998,193.580002,193.580002,34049900,153.720001,154.080002,152.949997,...,0.002125,0.002125,-0.031594,-0.031594,0.004036,0.004036,-0.014460,-0.014460,-0.003220,18368.579125


In [20]:
titres = [apple, amazon, berkshire, google, meta, microsoft, nvidia,
          tesla, unitedhealth, exxon]

In [21]:
def preprocess_timeseries(element):
    element["Date"] = pd.to_datetime(element["Date"])
    element = element[["Date","Close"]]
    element.set_index("Date", inplace = True)
    
    return(element)

In [22]:
for i in range(len(titres)) : 
    titres[i] = preprocess_timeseries(titres[i])

In [23]:
apple, amazon, berkshire, google, meta, microsoft, nvidia, tesla, unitedhealth, exxon = titres

## création du portefeuille

### TODO feature engineering

- volatilité du high/low sur les 10 derniers jours
- delta high/low sur 10 derniers jours
- rajouter des tags

In [25]:
for i in range(len(titres)):
    titres[i]['Return'] = titres[i]['Close'].pct_change()

portfolio_returns = pd.DataFrame()

In [26]:
for i in range(len(titres)):
    portfolio_returns[i] = titres[i]['Return']

portfolio_returns['Portfolio'] = portfolio_returns.mean(axis=1)

In [27]:
initial_capital = 1000

portfolio_returns['Portfolio Value'] = initial_capital * (1 + portfolio_returns['Portfolio']).cumprod()
portfolio_returns = portfolio_returns.iloc[1:]
print(portfolio_returns.head())

                   0         1         2         3         4         5  \
Date                                                                     
2013-01-03 -0.012622  0.004547  0.004507  0.000581 -0.008214 -0.013396   
2013-01-04 -0.027855  0.002592  0.002457  0.019760  0.035650 -0.018716   
2013-01-07 -0.005882  0.035925 -0.004262 -0.004363  0.022949 -0.001870   
2013-01-08  0.002691 -0.007748  0.003852 -0.001974 -0.012237 -0.005245   
2013-01-09 -0.015629 -0.000113 -0.005223  0.006573  0.052650  0.005650   

                   6         7         8         9  Portfolio  Portfolio Value  
Date                                                                            
2013-01-03  0.000786 -0.016685 -0.046755 -0.001804  -0.008906       991.094425  
2013-01-04  0.032993 -0.010642  0.001923  0.004630   0.004279       995.335697  
2013-01-07 -0.028897 -0.001744  0.000000 -0.011578   0.000028       995.363207  
2013-01-08 -0.021926 -0.019220 -0.013246  0.006255  -0.006880       988.5153

In [11]:
fig = go.Figure()

# Ajout de la série financière
fig.add_trace(go.Scatter(x = portfolio_returns.index, 
                         y = portfolio_returns['Portfolio'], 
                         mode = 'lines',
                         name ='Évolution financière ptf', 
                         fill = "tozeroy"))

# Mise à jour des titres
fig.update_layout(title="Évolution de la valeur de ptf",
                  xaxis_title='Date',
                  yaxis_title='Valeur')

# Affichage du graphique
fig.show()

## XGBoost

In [12]:
portfolio_returns.head()

,0,1,2,3,4,5,6,7,8,9,Portfolio,Portfolio Value
Date,,,,,,,,,,,,
2013-01-03,-0.012622,0.004547,0.004507,0.000581,-0.008214,-0.013396,0.000786,-0.016685,-0.046755,-0.001804,-0.008906,991.094425
2013-01-04,-0.027855,0.002592,0.002457,0.019760,0.035650,-0.018716,0.032993,-0.010642,0.001923,0.004630,0.004279,995.335697
2013-01-07,-0.005882,0.035925,-0.004262,-0.004363,0.022949,-0.001870,-0.028897,-0.001744,0.000000,-0.011578,0.000028,995.363207
2013-01-08,0.002691,-0.007748,0.003852,-0.001974,-0.012237,-0.005245,-0.021926,-0.019220,-0.013246,0.006255,-0.006880,988.515396
2013-01-09,-0.015629,-0.000113,-0.005223,0.006573,0.052650,0.005650,-0.022418,-0.001187,0.018872,-0.003843,0.003533,992.007954
